In [18]:
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass
from dateutil import parser, tz
from datetime import datetime, timedelta
# from zoneinfo import ZoneInfo

import ics
from enum import Enum

class EventType(Enum):
    EVENT = 1
    COURSE = 2

@dataclass
class HugoEvent:
    kind: EventType = None
    title: str = None
    url: str = None
    content: str = None
    start: str = None
    end: str = None
    status: bool = False
    course_format: str = None
    length: int = 0
    level: str = None
    tags: list = None
    
def parse_hugo_event(event_html : str)->Event:
    title = event_html.find("h2", class_="entry-title").text
    url = event_html.find("h2", class_="entry-title").find("a").get("href")
    content = event_html.find('div', class_='entry-content').text.strip()
    raw_date = event_html.find('div', class_='entry-meta').text.replace('\t','').replace('\n','')
    date = datetime.strptime(raw_date.split(" at ")[0], "%B %d").replace(year=datetime.now().year)
    start = datetime.strptime(raw_date.split(" at ")[1].split(" - ")[0], "%I:%M%p").replace(year=date.year, month=date.month, day=date.day, tzinfo=tz.gettz("America/Los_Angeles"))
    end = datetime.strptime(raw_date.split(" at ")[1].split(" - ")[1][:-2], "%I:%M%p").replace(year=date.year, month=date.month, day=date.day, tzinfo=tz.gettz("America/Los_Angeles"))
    # time = event_html.find('div', class_='entry-meta').text.replace('\t','').replace('\n','')
    # start = time
    # end = event_html.find('div', class_='entry-meta').text.replace('\t','').replace('\n','').split(' - ')[1]
    
    return HugoEvent(EventType.EVENT, title, url, content, start.isoformat(), end.isoformat())

def parse_catalog_entry(class_html:str)->HugoCourse:
    title = class_html.find("h3", class_="entry-title").text
    url = class_html.find("a", class_="link").get("href")
    status = class_html.find("div", class_="status").text.strip().find("Full") < 1
    content = class_html.find("div", class_="entry-content").text.strip()
    time = class_html.find('div', class_='start').text.strip().replace('\t','').replace('\n','')
    start = time.split(":",1)[1].strip()[:-2]
    start = datetime.strptime(start, "%B %d, %Y, %I:%M%p").astimezone(tz.tzlocal())
    end = start + timedelta(hours=2)
    course_format = class_html.find('div', class_='format').text.strip().replace('\t','').replace('\n','').split(":")[1].strip()
    length = class_html.find('div', class_='length').text.strip().replace('\t','').replace('\n','').split(":")[1].strip()
    level = class_html.find('div', class_='level').text.strip().replace('\t','').replace('\n','').split(":")[1].strip()
    tags = class_html.find('div', class_='genre').text.strip().replace('\t','').replace('\n','').split(", ")
    
    return HugoEvent(EventType.COURSE, title, url, content, start.isoformat(), end.isoformat(), status, course_format, length, level, tags)

def lookup_hugo_calendar():
    # Go through all the pages until there is nothing on the next page
    page_number = 0
    hugo_events = []
    while True:
        page = BeautifulSoup(requests.get(f"https://hugohouse.org/courses/page/{page_number}/".format(page_number=page_number)).content, "html.parser")
        courses_in_page = page.find_all("div", class_="course")
        if len(courses_in_page) == 0:
            break
        hugo_events.append([parse_catalog_entry(c) for c in courses_in_page])
        page_number += 1
    
    # Get all of the upcoming events calenar
    events_url = "https://hugohouse.org/programs-events/"
    events_page = BeautifulSoup(requests.get(events_url).content, "html.parser")
    events_html = events_page.find_all("div", class_="event")
    hugo_events.append([parse_hugo_event(e) for e in events_html])
    
    return [item for sublist in hugo_events for item in sublist]



In [19]:

events = lookup_hugo_calendar()

In [20]:
events

[HugoEvent(kind=<EventType.COURSE: 2>, title='Writer’s Welcome Kit (ASYNCHRONOUS)', url='https://hugohouse.org/product/writers-welcome-kit-asynchronous-2/', content='This asynchronous course features online workbooks, including ten essential templates in Word and Excel to kickstart your writing life, and resources professional writers use to get published.', start='2023-04-09T00:00:00-07:00', end='2023-04-09T02:00:00-07:00', status=True, course_format='Online Asynchronous', length='', level='Open to all levels', tags=['Nonfiction', 'Poetry', 'The Writing Life', 'Memoir', 'Novel', 'Short Story', 'Essay', 'Fiction']),
 HugoEvent(kind=<EventType.COURSE: 2>, title='Freelance Going Pro (ASYNCHRONOUS)', url='https://hugohouse.org/product/freelance-going-pro-asynchronous-2/', content='Through guided and practical exercises, this course will give you the confidence to turn your art into a long-term, successful business.', start='2023-04-09T00:00:00-07:00', end='2023-04-09T02:00:00-07:00', stat

## Ical part


In [4]:
def add_hugo_event(hugo: , calendar : ics.Calendar):
    event = ics.Event()
    event.name = hugo.title
    event.begin = datetime.fromisoformat(hugo.start)
    event.end = datetime.fromisoformat(hugo.end)
    event.description = hugo.content
    calendar.events.add(event)


In [5]:
hugo_ics = ics.Calendar()

[add_hugo_event(course, hugo_ics) for course in hugo_events]

with open("my.ics", "w") as f:
    f.write(hugo_ics.serialize())

In [ ]:
from datetime import datetime

datetime.strptime("April 8 at 1:00pm" , "%B %d at %I:%M%p")

datetime.strptime(events[0].find("h2", class_="entry-title").find("a").get("href")[-11:] , "%Y-%m-%d/")
duration = events[0].find('div', class_='entry-meta').text.replace('\t','').replace('\n','').split(' at ')[1].split(' - ')
datetime.strptime(events[0].find("h2", class_="entry-title").find("a").get("href")[-11:] , "%Y-%m-%d/")
datetime.strptime(duration[0] , "%I:%M%p").time()
datetime.strptime(duration[1] , "%I:%M%p").time()
from dateutil import parser, tz
from datetime import datetime

parser.parse(events[0].find('div', class_='entry-meta').text.replace('\t','').replace('\n','').split(' - ')[0]).astimezone(tz.tzlocal())


In [16]:
d = datetime.strptime(hugo_events[0].start.split(" at ")[0], "%B %d").replace(year=datetime.now().year)
s = datetime.strptime(hugo_events[0].start.split(" at ")[1].split(" - ")[0], "%I:%M%p").replace(year=d.year, month=d.month, day=d.day).astimezone(tz.tzlocal())
e = datetime.strptime(hugo_events[0].start.split(" at ")[1].split(" - ")[1][:-2], "%I:%M%p").replace(year=d.year, month=d.month, day=d.day).astimezone(tz.tzlocal())
print([s, e])

[datetime.datetime(2023, 4, 8, 13, 0), datetime.datetime(2023, 4, 8, 14, 0)]


In [13]:
datetime.strptime(hugo_events[0].start.split(" at ")[1].split(" - ")[0], "%I:%M%p").replace(year=datetime.now().year)
# hugo_events[0].start.split(" at ")[1].split(" - ")[0]

datetime.datetime(2023, 1, 1, 13, 0)